# Pipelines
Une compétence essentielle pour déployer (et même tester) des modèles complexes avec prétraitement.

Dans ce tutoriel, vous apprendrez à utiliser les pipelines pour simplifier votre code de modélisation.

## Introduction
Les pipelines sont un moyen simple d'organiser votre code de prétraitement des données et de modélisation. Concrètement, un pipeline regroupe les étapes de prétraitement et de modélisation, de sorte que vous puissiez utiliser l'ensemble comme une seule étape.

De nombreux **data scientists** créent des modèles sans pipelines, mais les pipelines offrent d'importants avantages :

- **Code plus propre** : Gérer les données à chaque étape de prétraitement peut devenir compliqué. Avec un pipeline, vous n'aurez pas besoin de suivre manuellement vos données d'entraînement et de validation à chaque étape.
- **Moins de bugs** : Il y a moins de risques d'appliquer une étape de manière incorrecte ou d'oublier une étape de prétraitement.
- **Facilité de mise en production** : Passer d'un prototype à un modèle déployable à grande échelle peut être étonnamment difficile. Bien que nous n'abordions pas tous les aspects connexes ici, les pipelines peuvent faciliter cette transition.
- **Plus d'options pour la validation des modèles** : Vous verrez un exemple dans le prochain tutoriel, qui couvre la validation croisée.

### Exemple
Comme dans le tutoriel précédent, nous travaillerons avec le jeu de données sur le logement à Melbourne.

Nous ne nous concentrerons pas sur l'étape de chargement des données. Imaginez que vous avez déjà les données d'entraînement et de validation dans `X_train` , `X_valid` , `y_train` et `y_valid`.

Nous jetons un coup d'œil aux données d'entraînement avec la méthode head(). 

Remarquez que les données contiennent à la fois des données catégoriques et des colonnes avec des valeurs manquantes. Avec un pipeline, il est facile de traiter les deux !

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Charger les données
melbourne_file_path = 'melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

# Supprimer les lignes avec une cible manquante, séparer la cible des prédicteurs
melbourne_data.dropna(axis=0, subset=['Price'], inplace=True)
y = melbourne_data['Price']
X = melbourne_data.drop(['Price'], axis=1)  # Retirez inplace=True

# Identifier les colonnes numériques et catégoriques
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Variables catégorielles à conserver
keep_cols = ['Type', 'Method', 'Regionname']

# Supprimer les autres variables catégorielles
X = X.drop(columns=[col for col in categorical_cols if col not in keep_cols])

# Diviser en données d'entraînement et de validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Vérifier les dimensions
print(f"Taille de X_train : {X_train.shape}")
print(f"Taille de X_valid : {X_valid.shape}")
print(f"Taille de y_train : {y_train.shape}")
print(f"Taille de y_valid : {y_valid.shape}")


Taille de X_train : (10864, 15)
Taille de X_valid : (2716, 15)
Taille de y_train : (10864,)
Taille de y_valid : (2716,)


In [30]:
X_train.head()

,Rooms,Type,Method,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Regionname,Propertycount
12167,1,u,S,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,Southern Metropolitan,13240.0
6524,2,h,SA,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,Western Metropolitan,6380.0
8413,3,h,S,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,Western Metropolitan,3755.0
2919,3,u,SP,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,Northern Metropolitan,8870.0
6043,3,h,S,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,Western Metropolitan,4217.0


Nous construisons le pipeline complet en trois étapes :

---

### Étape 1 : Définir les étapes de prétraitement
Comme un pipeline regroupe les étapes de prétraitement et de modélisation, nous utilisons la classe ColumnTransformer pour regrouper différentes étapes de prétraitement. Le code ci-dessous :

- Impute les valeurs manquantes dans les données numériques, et
- Impute les valeurs manquantes et applique un encodage one-hot aux données catégoriques.

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Colonnes numériques et catégoriques
numerical_cols = ['Rooms', 'Distance', 'Landsize', 'BuildingArea']
categorical_cols = ['Type', 'Method', 'Regionname']

# Prétraitement des données numériques
numerical_transformer = SimpleImputer(strategy='mean')

# Prétraitement des données catégoriques
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Regrouper le prétraitement des données numériques et catégoriques
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


### Étape 2 : Définir le modèle
Ensuite, nous définissons un modèle de **forêt aléatoire** avec la classe bien connue RandomForestRegressor.

In [37]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

---

### Étape 3 : Créer et évaluer le pipeline
Enfin, nous utilisons la classe Pipeline pour définir un pipeline qui regroupe les étapes de prétraitement et de modélisation.

Points importants :

- Avec le pipeline, nous prétraitons les données d'entraînement et ajustons le modèle en une seule ligne de code. (Sans pipeline, il faudrait effectuer l'imputation, l'encodage one-hot et l'entraînement du modèle en étapes séparées, ce qui est particulièrement complexe avec des variables numériques et catégoriques !)
- Avec le pipeline, nous fournissons les caractéristiques non traitées dans **X_valid** à la commande **predict()**, et le pipeline prétraite automatiquement les caractéristiques avant de générer des prédictions. (Sans pipeline, il faudrait se souvenir de prétraiter les données de validation avant de faire des prédictions.)

In [40]:
from sklearn.metrics import mean_absolute_error

# Regrouper le prétraitement et la modélisation dans un pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Prétraitement des données d'entraînement, ajustement du modèle
my_pipeline.fit(X_train, y_train)

# Prétraitement des données de validation, génération des prédictions
preds = my_pipeline.predict(X_valid)

# Évaluer le modèle
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)


MAE: 193793.14743097773


---

### Conclusion
Les pipelines sont utiles pour organiser le code de **machine learning** et éviter les erreurs. Ils sont particulièrement précieux pour les **workflows** impliquant un prétraitement sophistiqué des données.





---

# Validation croisée

In [43]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

# Pipeline de modélisation
my_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

# Obtenir les scores de validation croisée
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("Scores MAE :\n", scores)
print("Score MAE moyen (sur les expériences) :", scores.mean())


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 406, in _fit
    X, fitted_transformer =